# ML Pipeline Preparation
Follow the instructions below to help you create your ML pipeline.
### 1. Import libraries and load data from database.
- Import Python libraries
- Load dataset from database with [`read_sql_table`](https://pandas.pydata.org/pandas-docs/stable/generated/pandas.read_sql_table.html)
- Define feature and target variables X and Y

In [ ]:
# Download necessary NLTK data
import nltk
nltk.download(['punkt', 'wordnet', 'averaged_perceptron_tagger'])

import re
import os
import numpy as np
import pickle
from numpy import asarray
from numpy import savetxt
import pandas as pd
from sqlalchemy import create_engine
from nltk.tokenize import word_tokenize
from nltk.stem import WordNetLemmatizer

from sklearn.pipeline import Pipeline
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier, AdaBoostClassifier
from sklearn.feature_extraction.text import CountVectorizer, TfidfTransformer
from sklearn.metrics import classification_report
from sklearn.multioutput import MultiOutputClassifier
from sklearn.neighbors import KNeighborsClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.model_selection import GridSearchCV
from sklearn.tree import DecisionTreeClassifier

In [ ]:
# load data from database
cwd = os.getcwd() # gets current working directory
dbwd = cwd.replace('/models','/data/').replace('\\models','\\data\\') 
engine = create_engine('sqlite:///'+dbwd+'DisasterResponse.db')

In [ ]:
df = pd.read_sql("SELECT * FROM Messages_Categories_Cleaned", engine)
df.head(3)

In [ ]:
df.info()

In [ ]:
# Target variable is message
X = df.message.values
X
#X.shape

In [ ]:
# Feature variables are all the categories
# Categories start from column 5 onwards
y = df.iloc[:,4:].values
y[1]
# y.shape

### 2. Write a tokenization function to process your text data

In [ ]:
def tokenize(text):
      
    # URL detection; replace URLs with urlplaceholder
    url_regex = 'http[s]?://(?:[a-zA-Z]|[0-9]|[$-_@.&+]|[!*\(\),]|(?:%[0-9a-fA-F][0-9a-fA-F]))+'
    detected_urls = re.findall(url_regex, text)
    for url in detected_urls:
        text = text.replace(url, "urlplaceholder")
    
    # Tokenize text
    tokens = word_tokenize(text)
    
    # Initiate lematizer
    lemmatizer = WordNetLemmatizer()

    # iterate through each token
    clean_tokens = []
    for tok in tokens:
        
        # lemmatize, normalize case, and remove leading/trailing white space
        clean_tok = lemmatizer.lemmatize(tok).lower().strip()
        clean_tokens.append(clean_tok)

    return clean_tokens

### ML Workflow

In [ ]:
def display_results(y_test, y_pred):
    labels = np.unique(y_pred, axis=0)
    accuracy = (y_pred == y_test).mean().mean()
        
    print("Labels:", labels)
    print("Accuracy:", accuracy)

In [ ]:
def main():
    # Load Data
    X_train, X_test, y_train, y_test = train_test_split(X, y)
    
    #Instantiate Transformers and Classifier
    vect = CountVectorizer(tokenizer=tokenize)
    tfidf = TfidfTransformer()
    clf = MultiOutputClassifier(RandomForestClassifier())

    # Fit and/or Transform each to the data
    X_train_counts = vect.fit_transform(X_train)
    X_train_tfidf = tfidf.fit_transform(X_train_counts)
    clf.fit(X_train_tfidf, y_train)

    # Predict on test data
    X_test_counts = vect.transform(X_test)
    X_test_tfidf = tfidf.transform(X_test_counts)
    y_pred = clf.predict(X_test_tfidf)

    # display results
    display_results(y_test, y_pred)

main()

### 3. Build a machine learning pipeline
This machine pipeline should take in the `message` column as input and output classification results on the other 36 categories in the dataset. You may find the [MultiOutputClassifier](http://scikit-learn.org/stable/modules/generated/sklearn.multioutput.MultiOutputClassifier.html) helpful for predicting multiple target variables.

In [ ]:
pipeline = Pipeline([
        ('vect', CountVectorizer(tokenizer=tokenize)),
        ('tfidf', TfidfTransformer()),
        ('clf', MultiOutputClassifier(RandomForestClassifier()))
    ])

### 4. Train pipeline
- Split data into train and test sets
- Train pipeline

In [ ]:
X_train, X_test, y_train, y_test = train_test_split(X, y)

In [ ]:
pipeline.fit(X_train, y_train)

In [ ]:
y_pred = pipeline.predict(X_test)

### 5. Test your model
Report the f1 score, precision and recall for each output category of the dataset. You can do this by iterating through the columns and calling sklearn's `classification_report` on each.

https://scikit-learn.org/stable/modules/model_evaluation.html#classification-report

In [ ]:
column_names = list(df.columns.values.tolist())
target_names = column_names[4:40]
print(target_names)

In [ ]:
targets = np.vstack(y_test)
pred_classes = np.vstack(y_pred)

In [ ]:
count = 0

for col1, col2 in zip(targets.T, pred_classes.T):
    # print("col1: ", col1)
    # print("col2: ", col2)
    print("Column: ", target_names[count])
    print("================================")
    print(classification_report(y_true=col1, y_pred=col2))
    print("================================")
    count += 1

### 6. Improve your model
Use grid search to find better parameters. 

In [ ]:
pipeline.get_params()

In [ ]:
# Custom Transformer
class StartingVerbExtractor(BaseEstimator, TransformerMixin):

    def starting_verb(self, text):
        sentence_list = nltk.sent_tokenize(text)
        for sentence in sentence_list:
            pos_tags = nltk.pos_tag(tokenize(sentence))
            first_word, first_tag = pos_tags[0]
            if first_tag in ['VB', 'VBP'] or first_word == 'RT':
                return True
        return False

    def fit(self, x, y=None):
        return self

    def transform(self, X):
        X_tagged = pd.Series(X).apply(self.starting_verb)
        return pd.DataFrame(X_tagged)

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(RandomForestClassifier()))
])

# specify parameters for grid search
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

### 7. Test your model
Show the accuracy, precision, and recall of the tuned model.  

Since this project focuses on code quality, process, and  pipelines, there is no minimum performance metric needed to pass. However, make sure to fine tune your models for accuracy, precision and recall to make your project stand out - especially for your portfolio!

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
y_pred = cv.predict(X_test)

In [ ]:
column_names = list(df.columns.values.tolist())
target_names = column_names[4:40]
print(target_names)

In [ ]:
targets = np.vstack(y_test)
pred_classes = np.vstack(y_pred)

In [ ]:
count = 0

for col1, col2 in zip(targets.T, pred_classes.T):
    # print("col1: ", col1)
    # print("col2: ", col2)
    print("Column: ", target_names[count])
    print("================================")
    print(classification_report(y_true=col1, y_pred=col2))
    print("================================")
    count += 1

### 8. Try improving your model further. Here are a few ideas:
* try other machine learning algorithms
* add other features besides the TF-IDF

In [ ]:
# Try MultiOutputClassifier(DecisionTreeClassifier()) and MultiOutputClassifier(AdaBoostClassifier())

'''
Try parameters if available:
- 'tfidf__max_df': 1.0,
- 'tfidf__max_features': None,
- 'tfidf__min_df': 1,
- 'tfidf__ngram_range': (1, 1),
- 'clf__estimator__learning_rate': 1.0,
- 'clf__estimator__n_estimators': 50,
'''

**Trying MultiOutputClassifier(DecisionTreeClassifier())**

In [ ]:
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(DecisionTreeClassifier()))
])

# specify parameters for grid search
parameters = {'clf__estimator__max_depth': [10, 50, None],
              'features__text_pipeline__vect__ngram_range': ((1, 1), (1, 2))}

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
column_names = list(df.columns.values.tolist())
target_names = column_names[4:40]

targets = np.vstack(y_test)
pred_classes = np.vstack(y_pred)

In [ ]:
count = 0

for col1, col2 in zip(targets.T, pred_classes.T):
    # print("col1: ", col1)
    # print("col2: ", col2)
    print("Column: ", target_names[count])
    print("================================")
    print(classification_report(y_true=col1, y_pred=col2))
    print("================================")
    count += 1

**Trying MultiOutputClassifier(AdaBoostClassifier())**

In [ ]:
pipeline.get_params()

In [ ]:
# Try MultiOutputClassifier(AdaBoostClassifier())
pipeline = Pipeline([
    ('features', FeatureUnion([

        ('text_pipeline', Pipeline([
            ('vect', CountVectorizer(tokenizer=tokenize)),
            ('tfidf', TfidfTransformer())
        ])),

        ('starting_verb', StartingVerbExtractor())
    ])),

    ('clf', MultiOutputClassifier(AdaBoostClassifier()))
])

# specify parameters for grid search
parameters = {
              'clf__estimator__learning_rate': [0.1, 0.2, 0.5, 1],
              'clf__estimator__n_estimators': [10, 20, 50]
             }

# create grid search object
cv = GridSearchCV(pipeline, param_grid=parameters, verbose=1)

In [ ]:
cv.fit(X_train, y_train)

In [ ]:
column_names = list(df.columns.values.tolist())
target_names = column_names[4:40]

targets = np.vstack(y_test)
pred_classes = np.vstack(y_pred)

In [ ]:
count = 0

for col1, col2 in zip(targets.T, pred_classes.T):
    # print("col1: ", col1)
    # print("col2: ", col2)
    print("Column: ", target_names[count])
    print("================================")
    print(classification_report(y_true=col1, y_pred=col2))
    print("================================")
    count += 1

### 9. Export your model as a pickle file

In [ ]:
# save the model to disk
filename = 'classifier.pkl'
pickle.dump(cv, open(filename, 'wb'))
 
# some time later...
 
# load the model from disk
# loaded_model = pickle.load(open(filename, 'rb'))
# result = loaded_model.score(X_test, Y_test)

### 10. Use this notebook to complete `train.py`
Use the template file attached in the Resources folder to write a script that runs the steps above to create a database and export a model based on a new dataset specified by the user.